<a href="https://colab.research.google.com/github/mcdona20/Xtern/blob/main/Xtern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project, Point of Interest (POI) data was collected from OpenStreetMap using Overpass Turbo. I decided to focus on the downtown area and use a night game at Victory Field (beginning at 7 PM) as the end point for the day. 
---
Restaurants, cafes, and ice cream shops were queried within 1 km of the Soldiers' and Sailors' Monument. Tourism sites (zoos, galleries, attractions) were queried within 2 km. Starting and ending the day at a bike rental would be a nice option for transportation, so bike rentals were queried as well.


In [ ]:
# Import location and plan datasets in Pandas Dataframes

import pandas as pd
url = 'https://raw.githubusercontent.com/mcdona20/Xtern/main/Location%20Data.csv'
df1 = pd.read_csv(url)
url = 'https://raw.githubusercontent.com/mcdona20/Xtern/main/Plan.csv'
plan_df = pd.read_csv(url)


Below is a map of restaurants and cafes within the search radius. The area near Victory Field will be the beginning and the end point of the day, for simplicity and because of ample parking in the area. The bike share station next to Victory Field will provide transportation to the students. Only this bike share has been marked, though there are several others within the queried area. 

In [ ]:
# Plot map of queried cafes and restaurants

import folium
Center_coordinates = [39.768507507898114, -86.1580388607257]
food_map=folium.Map(location= Center_coordinates, zoom_start=15)
folium.Circle(Center_coordinates,radius=1000).add_to(food_map)

loc = 'Downtown Indy, Food Within 1 km Radius of Center'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
food_map.get_root().html.add_child(folium.Element(title_html))

# Plot each individual establishment marker
for index, series in df1.iterrows():
  if series['amenity'] == 'cafe':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="black",icon="coffee", prefix='fa')).add_to(food_map)
  if series['amenity'] == 'restaurant':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="blue",icon="cutlery", prefix='fa')).add_to(food_map)
folium.Marker (location=[39.7647255,-86.168636], icon = folium.Icon(color="red",icon="trophy", prefix='fa')).add_to(food_map)
folium.Marker (location=[39.7659457,-86.1668074], icon = folium.Icon(color="green",icon="bicycle", prefix='fa')).add_to(food_map)


food_map


Below is a map of attractions (blue) and museums (black) within the 2 km search radius. We will focus on the area closest to Victory Field, since this day is intended to be walkable/bikeable. The Eiteljorg Museum in the museum complex just north of Victory Field will be a starting point, and the Indianapolis Zoo across the White River will fill in afternoon time.

In [ ]:
# Plot map of queried attractions and museums


Center_coordinates = [39.768507507898114, -86.1580388607257]
attraction_map=folium.Map(location= Center_coordinates, zoom_start=15)
folium.Circle(Center_coordinates,radius=2000).add_to(attraction_map)

loc = 'Downtown Indy, Attractions Within 2 km Radius of Center'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
attraction_map.get_root().html.add_child(folium.Element(title_html))

# Plot each individual establishment marker
for index, series in df1.iterrows():
  if series['tourism'] == 'attraction':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="blue",icon="flag", prefix='fa')).add_to(attraction_map)
  if series['tourism'] == 'museum':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="black",icon="institution", prefix='fa')).add_to(attraction_map)
folium.Marker (location=[39.7647255,-86.168636], icon = folium.Icon(color="red",icon="trophy", prefix='fa')).add_to(attraction_map)
folium.Marker (location=[39.7659457,-86.1668074], icon = folium.Icon(color="green",icon="bicycle", prefix='fa')).add_to(attraction_map)


attraction_map

Below is the planned day and then a map of our chosen locations for the day. [Cafe Patachou](https://www.cafepatachou.com/) will provide lunch, and [St. Elmo's Steakhouse](https://www.stelmos.com/) will provide dinner. The [Eiteljorg Museum](https://eiteljorg.org/), the [Indianapolis Zoo](https://www.indianapoliszoo.com/), and [Victory Field](https://www.milb.com/indianapolis/ballpark/victory-field) will be the main activity sites. These locations are all within quick biking distance of each other, for a compact, no-driving day. Biking routes (in red) were exported out of GrassHopper, with the longest single route (between the Zoo and St. Elmo's) taking 16 minutes of riding. 

In [ ]:
print(plan_df)

       Time                     Name  \
0   9:00 AM         Eiteljorg Museum   
1  12:00 PM            Cafe Patechou   
2   1:00 PM  Victory Field Bikeshare   
3   1:30 PM         Indianapolis Zoo   
4   5:00 PM    St. Elmo's Steakhouse   
5   7:00 PM            Victory Field   

                                        Address  Activity_Type   Duration  
0   500 W Washington St, Indianapolis, IN 46204           Tour    3 hours  
1   225 W Washington St, Indianapolis, IN 46204          Lunch     1 hour  
2           Corner of Maryland and West Streets    Bike Rental  0.5 hours  
3  1200 W Washington St, Indianapolis, IN 46222           Tour  3.5 hours  
4     127 S Illinois St, Indianapolis, IN 46225       Shopping    2 hours  
5     501 W Maryland St, Indianapolis, IN 46225  Baseball Game    3 hours  


In [ ]:
# Plot map of chosen locations for the day with bike routes between them


!pip install gpxpy
import gpxpy.gpx
!wget https://raw.githubusercontent.com/mcdona20/Xtern/main/GraphHopper.gpx # bike route directions exported from GraphHopper
gpx_file = open('GraphHopper.gpx', 'r')

gpx = gpxpy.parse(gpx_file)
points = []
for track in gpx.tracks:
    for segment in track.segments:        
        for point in segment.points:
            points.append(tuple([point.latitude, point.longitude]))


ave_lat = sum(p[0] for p in points)/len(points)
ave_lon = sum(p[1] for p in points)/len(points)
 
# Load map centered on average coordinates
directions_map = folium.Map(location=[ave_lat, ave_lon], zoom_start=16)
loc = 'Chosen Locations for Day, with Biking Route'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
directions_map.get_root().html.add_child(folium.Element(title_html))

# add lines for bike route
folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(directions_map)

# add markers for selected locations
for index, series in df1.iterrows():
  if series['name'] == 'Eiteljorg Museum':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="black",icon="institution", prefix='fa')).add_to(directions_map)
  if series['name'] == 'Indianapolis Zoo':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="black",icon="flag", prefix='fa')).add_to(directions_map)
  if series['name'] == 'St. Elmo Steak House':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="blue",icon="cutlery", prefix='fa')).add_to(directions_map)
  if series['name'] == 'Cafe Patachou':
    lat = series['@lat']
    lng = series['@lon']
    folium.Marker (location=[lat,lng], icon = folium.Icon(color="blue",icon="cutlery", prefix='fa')).add_to(directions_map)  
folium.Marker (location=[39.7647255,-86.168636], icon = folium.Icon(color="red",icon="trophy", prefix='fa')).add_to(directions_map)
folium.Marker (location=[39.7659457,-86.1668074], icon = folium.Icon(color="green",icon="bicycle", prefix='fa')).add_to(directions_map)

directions_map

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-10-17 16:49:30--  https://raw.githubusercontent.com/mcdona20/Xtern/main/GraphHopper.gpx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18437 (18K) [text/plain]
Saving to: ‘GraphHopper.gpx.18’

GraphHopper.gpx.18  100%[===================>]  18.00K  --.-KB/s    in 0.001s  

2022-10-17 16:49:30 (24.3 MB/s) - ‘GraphHopper.gpx.18’ saved [18437/18437]

